In [16]:
import nltk
import random
from nltk.tokenize import word_tokenize

from nltk.corpus import movie_reviews

import pickle

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode 

In [17]:
class VoteClassifier(ClassifierI):
    
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [18]:
short_pos = open("short_reviews/positive.txt","rb").read()
short_neg = open("short_reviews/negative.txt","rb").read()
print(type(short_neg))

<class 'bytes'>


In [19]:
documents = []

for r in short_pos.split(b"\n"):
    documents.append((str(r), "pos"))

for r in short_neg.split(b"\n"):
    documents.append((str(r), "neg"))

all_words = []
short_pos_words = word_tokenize(str(short_pos))
short_neg_words = word_tokenize(str(short_neg))

for w in short_pos_words:
    all_words.append(w.lower())
for w in short_neg_words:
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

In [20]:
word_features = list(all_words.keys())[:5000]

In [21]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words) 
    return features

In [22]:
#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))

In [23]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

In [24]:
training_set = featuresets[:10000] 
testing_set = featuresets[10000:]

In [25]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
#classifier_f = open("naivebayes.pickle","rb")
#classifier = pickle.load(classifier_f)
#classifier_f.close()
print("Original Naive Bayes Algo Accuracy : ",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Original Naive Bayes Algo Accuracy :  74.09638554216868
Most Informative Features
              engrossing = True              pos : neg    =     19.9 : 1.0
                provides = True              pos : neg    =     16.5 : 1.0
                  stupid = True              neg : pos    =     16.2 : 1.0
                  unique = True              pos : neg    =     14.5 : 1.0
              refreshing = True              pos : neg    =     13.8 : 1.0
               inventive = True              pos : neg    =     13.1 : 1.0
            refreshingly = True              pos : neg    =     11.8 : 1.0
             mesmerizing = True              pos : neg    =     11.1 : 1.0
                touching = True              pos : neg    =     11.1 : 1.0
                    ages = True              pos : neg    =     10.4 : 1.0
                    warm = True              pos : neg    =     10.3 : 1.0
               wonderful = True              pos : neg    =     10.3 : 1.0
                ca

In [26]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_Classifier Algo Accuracy : ",(nltk.classify.accuracy(MNB_classifier, testing_set))*100)

MNB_Classifier Algo Accuracy :  74.24698795180723


In [27]:
#GaussianNB_classifier = SklearnClassifier(GaussianNB())
#GaussianNB_classifier.train(training_set)
#print("GaussianNB_classifier Algo Accuracy : ",(nltk.classify.accuracy(GaussianNB_classifier, testing_set))*100)

In [28]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier Algo Accuracy : ",(nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

BernoulliNB_classifier Algo Accuracy :  73.49397590361446


In [29]:
#LogisticRegression, SGDClassifier
#SVC, LinearSVC, NuSVC

In [30]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier Algo Accuracy : ",(nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

LogisticRegression_classifier Algo Accuracy :  73.49397590361446


In [31]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier Algo Accuracy : ",(nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

SGDClassifier_classifier Algo Accuracy :  66.41566265060241


In [32]:
#SVC_classifier = SklearnClassifier(SVC())
#SVC_classifier.train(training_set)
#print("SVC_classifier Algo Accuracy : ",(nltk.classify.accuracy(SVC_classifier, testing_set))*100)

In [33]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier Algo Accuracy : ",(nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

LinearSVC_classifier Algo Accuracy :  72.89156626506023


In [34]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier Algo Accuracy : ",(nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

NuSVC_classifier Algo Accuracy :  73.64457831325302


In [35]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier,
                                  SGDClassifier_classifier,
                                  LinearSVC_classifier,
                                  NuSVC_classifier)
print("voted_classifier Algo Accuracy : ",(nltk.classify.accuracy(voted_classifier, testing_set))*100)

print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:", voted_classifier.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:", voted_classifier.confidence(testing_set[1][0])*100)
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:", voted_classifier.confidence(testing_set[2][0])*100)
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:", voted_classifier.confidence(testing_set[3][0])*100)
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:", voted_classifier.confidence(testing_set[4][0])*100)
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:", voted_classifier.confidence(testing_set[5][0])*100)



voted_classifier Algo Accuracy :  74.69879518072288
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
